In [1]:
import numpy as np
import geopandas as gpd
import xarray as xr
import salem
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
from cartopy.io.shapereader import Reader, natural_earth
import matplotlib.pyplot as plt
import os

> with……as……表示打开一个文件后自动关闭

In [2]:
def GetVar(file_names, DatasetName):
    with  xr.open_dataset(file_names) as ncf:
        print(ncf)
        getvars = ncf[DatasetName][:,:]
        print("File closed")
        return getvars 

In [3]:
def CreateMap(fig):
    proj = ccrs.LambertConformal(central_longitude=110, central_latitude=35, 
                             standard_parallels=(30, 60))  # 创建坐标系

    ax = fig.subplots(1, 1, subplot_kw={'projection': proj})  
    # --设置shp
    provinces = cfeat.ShapelyFeature(
        Reader(shp_path).geometries(),
        ccrs.PlateCarree(), 
        edgecolor='k',
        facecolor='none',
    )
    ax.add_feature(provinces, linewidth=0.6, zorder=2)
    # --设置范围
    ax.set_extent([80, 135, 15, 55], crs=ccrs.PlateCarree())
    # --关闭边框
    ax.spines['geo'].set_visible(False)
    # --设置刻度
    ax.tick_params(
        axis='both', 
        labelsize=5, 
        direction='out',
        labelbottom=False, 
        labeltop=False, 
        labelleft=False, 
        labelright=False
    )
    # --设置小地图
    left, bottom, width, height = 0.675, 0.13, 0.23, 0.27
    ax2 = fig.add_axes(
        [left, bottom, width, height], 
        projection=ccrs.PlateCarree()
    )
    provinces = cfeat.ShapelyFeature(
        Reader(shp_path).geometries(),
        ccrs.PlateCarree(), 
        edgecolor='k',
        facecolor='gray',
    )
    ax2.add_feature(provinces, linewidth=0.6, zorder=2)
    ax2.set_extent([105, 125, 0, 25], crs=ccrs.PlateCarree())    
    return ax

> "china"是初始的默认参数，调用函数时，实参"Test"覆盖了默认的参数值（在函数调用中明确指定的实参优先级高于默认参数的值）。  
默认值用于在调用函数时省略对应的实参：

```python
MaskRegion(2)
```


In [6]:
def MaskRegion(ds, region='china'): #相当于将region赋值为‘China’，传入的是['江苏省']
    shp = gpd.read_file(shp_path).set_index('省')
    if region != 'china':
        shp = shp.loc[region, :] ###
    ds_mask = ds.salem.roi(shape=shp)
    return ds_mask

### 经测试和MaskRegion效果相同 [Geopandas.clip官方文档](https://geopandas.org/en/stable/gallery/plot_clip.html?highlight=clip)
> ###后面为不一样的语句

In [7]:
def MaskRegion2(ds):
    shp = gpd.read_file(shp_path).set_index('省')
    JS=gdf[gdf['省']=='江苏省'] ###
    a=gpd.clip(gdf, JS)
    ds_mask = ds.salem.roi(shape=a)
    return ds_mask

In [5]:
def DrawFig(ds, levels, cmap, outpath='.',savename='save.png'):
    fig = plt.figure(figsize=(10, 7), dpi=130)  # 创建页面
    
    im = ds.plot.contourf(
        ax=CreateMap(fig), 
        cmap=cmap, 
        transform=ccrs.PlateCarree(),
        levels=levels,
        add_colorbar=False
    )
    cbar_ax = fig.add_axes([0.13, 0.17, 0.3, 0.012])
    cb = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
    #plt.savefig(f'{outpath}{os.sep}maskmap.png')
    plt.savefig(savename)

> 如果执行的是此程序，__name__=__main__；如果引用的模块，__name__=程序名，则不运行下面程序，一般运用于测试

In [ ]:
#D:/study/MaskoutProvince/ExampleData.nc
#D:/study/MaskoutProvince/province.shp
if __name__ == "__main__":
    file_name2 = r"./ExampleData.nc"
    ds = GetVar(file_name2, "pre")
    levels = np.arange(0, 20, 2)
    cmap = 'Spectral_r'
    shp_path = r'./province.shp' 
    
    ds_mask = MaskRegion(ds,["江苏省"])
    print("ds_masked")
    
    DrawFig(ds, levels, cmap, savename="unmaskdata.png")
    DrawFig(ds_mask, levels, cmap, savename="maskdata.png")
    
    ds = np.array(ds)
    ds_mask = np.array(ds_mask)